In [3]:
# ========================================
# EXTRACT CHECKSUMS FROM ALL MESSAGES WITH CHECKSUM IN DBC
# ========================================

import pandas as pd
import cantools
from python.psa_checksum import psa_checksum  # Assuming this library is available in the environment

print("🚀 START CHECKSUM EXTRACTION FOR ALL MESSAGES\n")

# Configuration
# LOG_FILE = 'logs/aa0ad8ba95ff270c|00000017--62c22bb216.csv'  # Main log file
#LOG_FILE = 'logs/6a7075a4fdd765ee|0000004e--1f612006dd.csv'
LOG_FILE = 'logs/sunny1.csv'

DBC_FILE = 'dbc/psa_aee2010_r3.dbc'  # DBC file

# 1. LOAD DBC
print(f"📂 Loading DBC: {DBC_FILE}...")
try:
    dbc = cantools.database.load_file(DBC_FILE, strict=False)
    print(f"✅ DBC loaded\n")
except FileNotFoundError:
    print(f"❌ File {DBC_FILE} not found!")
    exit()

# 2. LOAD CSV (once, for all messages)
print(f"\n📂 Loading CSV: {LOG_FILE}...")
df = pd.read_csv(LOG_FILE)
print(f"✅ Loaded: {len(df)} rows\n")

# 3. FIND ALL MESSAGES WITH CHECKSUM IN DBC
messages_with_checksum = []
for msg in dbc.messages:
    checksum_signals = [sig for sig in msg.signals if 'CHECKSUM' in sig.name.upper() or 'CRC' in sig.name.upper()]
    if checksum_signals:
        messages_with_checksum.append((msg, checksum_signals))

print(f"✅ Found {len(messages_with_checksum)} messages with checksum in DBC\n")

# Initialize global statistics
global_stats = {
    'total_message_types': 0,
    'total_messages_analyzed': 0,
    'total_messages_ok': 0,
    'total_messages_fail': 0,
    'details_per_message': []
}

# 4. LOOP THROUGH EACH MESSAGE WITH CHECKSUM
for msg, checksum_signals in messages_with_checksum:
    MESSAGE_ID = msg.frame_id
    
    print("=" * 80)
    print(f"PROCESS MESSAGE 0x{MESSAGE_ID:03X} ({msg.name})")
    print("=" * 80)
    
    # Show message definition
    print(f"Name: {msg.name}")
    print(f"ID: 0x{MESSAGE_ID:03X} ({MESSAGE_ID})")
    print(f"Length: {msg.length} bytes")
    print(f"Signals: {len(msg.signals)}")
    print()
    
    print("📋 SIGNALS IN MESSAGE:")
    print("-" * 80)
    for sig in msg.signals:
        print(f"  {sig.name:30s} | Start: {sig.start:3d} | Length: {sig.length:2d} bit | Scale: {sig.scale} | Offset: {sig.offset}")
        if 'CHECKSUM' in sig.name.upper() or 'CRC' in sig.name.upper():
            print(f"     ⚡ THIS IS A CHECKSUM!")
    
    print()
    print(f"✅ Found {len(checksum_signals)} checksum:")
    for cs in checksum_signals:
        print(f"   - {cs.name}: bit {cs.start}, length {cs.length} bit")
    print()
    
    # 5. FIND THE CORRECT BUS FOR THIS MESSAGE
    # Special case: HS2_DAT_MDD_CMD_452 must use only bus 1
    if msg.name == "HS2_DAT_MDD_CMD_452":
        print(f"⚠️  Special message {msg.name}: forced use of bus 1")
        possible_buses = [1]  # Only bus 1 for this message
    else:
        possible_buses = [0, 1, 2, 3]  # Assume possible buses (expand if needed)
    
    BUS = None
    df_filtered = None
    for b in possible_buses:
        temp_filtered = df[(df['bus'] == b) & (df['addr'] == f'0x{MESSAGE_ID:x}')].copy()
        if len(temp_filtered) > 0:
            BUS = b
            df_filtered = temp_filtered
            break
    
    if BUS is None:
        print(f"❌ No message found for 0x{MESSAGE_ID:03X} on any bus!")
        continue
    
    print(f"🔍 Messages found on bus {BUS}: {len(df_filtered)}")
    print()
    
    # 6. DECODE AND EXTRACT CHECKSUM
    print("🔄 Decoding messages and extracting checksum...\n")
    
    decoded_messages = []
    for idx, row in df_filtered.iterrows():
        try:
            time = row['time']
            data_hex = row['data']
            
            # Prepare bytes
            if data_hex.startswith('0x'):
                data_hex = data_hex[2:]
            if len(data_hex) % 2:
                data_hex = '0' + data_hex
            data_bytes = bytes.fromhex(data_hex)
            
            # Decode
            decoded = dbc.decode_message(MESSAGE_ID, data_bytes)
            
            # Add basic info
            decoded['time'] = time
            decoded['raw_hex'] = data_hex
            decoded['raw_bytes'] = ' '.join([data_hex[i:i+2].upper() for i in range(0, len(data_hex), 2)])
            
            # Verify ALL checksums
            all_checksums_ok = True
            
            for cs_sig in checksum_signals:
                # Calculate checksum - CREATE FRESH COPY EVERY TIME
                data_array = bytearray(data_bytes)  # Fresh copy for each checksum!
                calculated = psa_checksum(MESSAGE_ID, cs_sig, data_array)
                
                # Extract checksum from message
                extracted = decoded.get(cs_sig.name, None)
                
                # Convert NamedSignalValue to int
                if hasattr(extracted, 'value'):
                    extracted = int(extracted.value)
                else:
                    extracted = int(extracted) if extracted is not None else 0
                    
                # Verify
                is_ok = (calculated == extracted)
                all_checksums_ok = all_checksums_ok and is_ok
                
                # Save with unique name for each checksum
                decoded[f'{cs_sig.name}_extracted'] = extracted
                decoded[f'{cs_sig.name}_calculated'] = calculated
                decoded[f'{cs_sig.name}_ok'] = is_ok
            
            # Global flag: OK only if ALL are OK
            decoded['all_checksums_ok'] = all_checksums_ok
            
            decoded_messages.append(decoded)
            
        except Exception as e:
            print(f"⚠️  Decode error at t={time:.3f}s: {e}")
    
    if not decoded_messages:
        print("❌ No messages decoded!")
        continue
    
    df_decoded = pd.DataFrame(decoded_messages)
    print(f"✅ Decoded {len(df_decoded)} messages\n")
    
    # 7. SHOW RESULTS (similar to original code, but abbreviated for brevity)
    print("=" * 80)
    print("MESSAGES WITH CHECKSUM")
    print("=" * 80)
    print()
    
    N = 5  # Show first 5
    print(f"📋 First {N} messages:\n")
    
    for idx, row in df_decoded.head(N).iterrows():
        time = row['time']
        raw_bytes = row['raw_bytes']
        all_ok = row['all_checksums_ok']
        
        # Global status
        status = "✓ OK" if all_ok else "✗ FAIL"
        
        print(f"[{time:8.3f}s] {status:6s} | {raw_bytes}")
        
        # Show each checksum on separate line
        for cs in checksum_signals:
            extr_val = row[f'{cs.name}_extracted']
            calc_val = row[f'{cs.name}_calculated']
            
            # Convert to int handling NamedSignalValue
            try:
                extr = int(float(extr_val))
            except:
                extr = int(extr_val.value) if hasattr(extr_val, 'value') else int(extr_val)
                
            try:
                calc = int(float(calc_val))
            except:
                calc = int(calc_val.value) if hasattr(calc_val, 'value') else int(calc_val)
            
            ok_symbol = "✓" if row[f'{cs.name}_ok'] else "✗"
            print(f"   {cs.name:30s}: Extracted=0x{extr:X}  Calculated=0x{calc:X}  [{ok_symbol}]")
        
        # Other signals (optional, abbreviated)
        print()
    
    # 8. CHECKSUM STATISTICS (similar to original code)
    print(f"\n{'=' * 80}")
    print("📊 CHECKSUM STATISTICS")
    print("=" * 80)
    
    all_ok_count = df_decoded['all_checksums_ok'].sum()
    all_fail_count = len(df_decoded) - all_ok_count
    
    print(f"Total messages:        {len(df_decoded)}")
    print(f"All checksums OK:      {all_ok_count} ({all_ok_count/len(df_decoded)*100:.1f}%)" if len(df_decoded) > 0 else "N/A")
    print(f"At least 1 FAIL:       {all_fail_count} ({all_fail_count/len(df_decoded)*100:.1f}%)" if len(df_decoded) > 0 else "N/A")
    print()
    
    # Statistics for each checksum
    for cs in checksum_signals:
        print(f"--- {cs.name} ---")
        ok_count = df_decoded[f'{cs.name}_ok'].sum()
        fail_count = len(df_decoded) - ok_count
        print(f"  OK:   {ok_count} ({ok_count/len(df_decoded)*100:.1f}%)" if len(df_decoded) > 0 else "N/A")
        print(f"  FAIL: {fail_count} ({fail_count/len(df_decoded)*100:.1f}%)" if len(df_decoded) > 0 else "N/A")
        print()
    
    if all_fail_count > 0:
        print(f"⚠️  Examples of messages with wrong checksums (first 5):")
        fails = df_decoded[~df_decoded['all_checksums_ok']].head(5)
        for idx, row in fails.iterrows():
            print(f"\n   t={row['time']:.3f}s: {row['raw_bytes']}")
            for cs in checksum_signals:
                if not row[f'{cs.name}_ok']:
                    extr_val = row[f'{cs.name}_extracted']
                    calc_val = row[f'{cs.name}_calculated']
                    # Try different ways to convert
                    try:
                        extr = int(float(extr_val))
                        calc = int(float(calc_val))
                    except:
                        extr = extr_val
                        calc = calc_val
                    print(f"      {cs.name}: Extracted={extr}, Calculated={calc}")
        print()
    
    # Update global statistics
    global_stats['total_message_types'] += 1
    global_stats['total_messages_analyzed'] += len(df_decoded)
    global_stats['total_messages_ok'] += all_ok_count
    global_stats['total_messages_fail'] += all_fail_count
    
    global_stats['details_per_message'].append({
        'message_id': f"0x{MESSAGE_ID:03X}",
        'message_name': msg.name,
        'bus': BUS,
        'total': len(df_decoded),
        'ok': all_ok_count,
        'fail': all_fail_count,
        'success_rate': (all_ok_count/len(df_decoded)*100) if len(df_decoded) > 0 else 0
    })


🚀 START CHECKSUM EXTRACTION FOR ALL MESSAGES

📂 Loading DBC: dbc/psa_aee2010_r3.dbc...
✅ DBC loaded


📂 Loading CSV: logs/sunny1.csv...
✅ Loaded: 854684 rows

✅ Found 10 messages with checksum in DBC

PROCESS MESSAGE 0x1CD (ESP)
Name: ESP
ID: 0x1CD (461)
Length: 3 bytes
Signals: 5

📋 SIGNALS IN MESSAGE:
--------------------------------------------------------------------------------
  COUNTER                        | Start:   3 | Length:  4 bit | Scale: 1 | Offset: 0
  CHECKSUM                       | Start:   4 | Length:  4 bit | Scale: 1 | Offset: 0
     ⚡ THIS IS A CHECKSUM!
  ESP_STATUS                     | Start:  15 | Length:  1 bit | Scale: 1 | Offset: 0
  UNCLEAR                        | Start:  14 | Length: 10 bit | Scale: 1 | Offset: 0
  ESP_STATUS_INV                 | Start:  20 | Length:  1 bit | Scale: 1 | Offset: 0

✅ Found 1 checksum:
   - CHECKSUM: bit 4, length 4 bit

❌ No message found for 0x1CD on any bus!
PROCESS MESSAGE 0x2B6 (HS2_DYN1_MDD_ETAT_2B6)
Name: HS2_DYN

In [4]:
# ========================================
# FINAL AGGREGATE STATISTICS
# ========================================
print("\n\n")
print("=" * 100)
print("🏁 FINAL STATISTICS - COMPLETE SUMMARY")
print("=" * 100)
print()

print(f"📊 OVERALL TOTALS:")
print(f"   Message types analyzed:      {global_stats['total_message_types']}")
print(f"   Total messages analyzed:     {global_stats['total_messages_analyzed']}")
print(f"   Messages with checksum OK:   {global_stats['total_messages_ok']} ({global_stats['total_messages_ok']/global_stats['total_messages_analyzed']*100:.2f}%)" if global_stats['total_messages_analyzed'] > 0 else "N/A")
print(f"   Messages with WRONG checksum: {global_stats['total_messages_fail']} ({global_stats['total_messages_fail']/global_stats['total_messages_analyzed']*100:.2f}%)" if global_stats['total_messages_analyzed'] > 0 else "N/A")
print()

print("=" * 100)
print("📋 DETAIL BY MESSAGE TYPE:")
print("=" * 100)
print()

# Sort by success rate (from worst to best)
sorted_details = sorted(global_stats['details_per_message'], key=lambda x: x['success_rate'])

for detail in sorted_details:
    status_icon = "✅" if detail['success_rate'] == 100.0 else "⚠️" if detail['success_rate'] > 0 else "❌"
    print(f"{status_icon} {detail['message_id']} ({detail['message_name']}) - Bus {detail['bus']}")
    print(f"   Total: {detail['total']:6d} | OK: {detail['ok']:6d} ({detail['success_rate']:6.2f}%) | FAIL: {detail['fail']:6d} ({100-detail['success_rate']:6.2f}%)")
    print()

# Summary of messages with problems
messages_with_errors = [d for d in global_stats['details_per_message'] if d['fail'] > 0]
messages_perfect = [d for d in global_stats['details_per_message'] if d['success_rate'] == 100.0]

print("=" * 100)
print("🎯 SUMMARY:")
print("=" * 100)
print(f"✅ Messages with PERFECT checksums (100%):  {len(messages_perfect)}/{global_stats['total_message_types']}")
print(f"⚠️  Messages with AT LEAST ONE ERROR:       {len(messages_with_errors)}/{global_stats['total_message_types']}")
print()

if messages_with_errors:
    print("⚠️  MESSAGES WITH ERRORS:")
    for detail in sorted(messages_with_errors, key=lambda x: x['fail'], reverse=True):
        print(f"   - {detail['message_id']} ({detail['message_name']}) Bus {detail['bus']}: {detail['fail']} errors out of {detail['total']} ({100-detail['success_rate']:.2f}%)")

print("\n🎉 COMPLETED!")




🏁 FINAL STATISTICS - COMPLETE SUMMARY

📊 OVERALL TOTALS:
   Message types analyzed:      9
   Total messages analyzed:     78709
   Messages with checksum OK:   69080 (87.77%)
   Messages with WRONG checksum: 9629 (12.23%)

📋 DETAIL BY MESSAGE TYPE:

❌ 0x3AD (Dyn_EasyMove) - Bus 0
   Total:   9483 | OK:      0 (  0.00%) | FAIL:   9483 (100.00%)

❌ 0x3F2 (LANE_KEEP_ASSIST) - Bus 0
   Total:    146 | OK:      0 (  0.00%) | FAIL:    146 (100.00%)

✅ 0x2B6 (HS2_DYN1_MDD_ETAT_2B6) - Bus 1
   Total:   9448 | OK:   9448 (100.00%) | FAIL:      0 (  0.00%)

✅ 0x2F5 (STEERING) - Bus 0
   Total:  18964 | OK:  18964 (100.00%) | FAIL:      0 (  0.00%)

✅ 0x2F6 (HS2_DYN_MDD_ETAT_2F6) - Bus 1
   Total:   9447 | OK:   9447 (100.00%) | FAIL:      0 (  0.00%)

✅ 0x305 (STEERING_ALT) - Bus 0
   Total:  18893 | OK:  18893 (100.00%) | FAIL:      0 (  0.00%)

✅ 0x38D (HS2_DYN_ABR_38D) - Bus 0
   Total:   4741 | OK:   4741 (100.00%) | FAIL:      0 (  0.00%)

✅ 0x42D (NEW_MSG_42D) - Bus 0
   Total:   3793 